In [42]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [24]:
a = torch.tril(torch.ones(3,3))
a = a / a.sum(1, keepdim=True)
print(a.shape)
a

torch.Size([3, 3])


tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

In [26]:
b2d = torch.tensor( [
       [4, 5],
       [7, 7], 
       [1, 1]
      ], dtype=torch.float)
print(b2d[0])
a @ b2d

tensor([4., 5.])


tensor([[4.0000, 5.0000],
        [5.5000, 6.0000],
        [4.0000, 4.3333]])

In [39]:
an = torch.tril(torch.ones(3, 3))
affin = torch.zeros(3, 3)
masked_affin = affin.masked_fill(an == 0, float('-inf'))
wei = F.softmax(masked_affin, 1)
wei

tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])

## The position embedding table

this seems rushed through, we create an embedding for positions - so for context length - instead of a dimensional embedding of vocab_size

In [47]:
n_embed = 3
vocab_size = 27

batch_size = 4
context_length = 8

In [45]:
vocab_embedding_table = nn.Embedding(vocab_size, n_embed)
position_embedding_table = nn.Embedding(context_length, n_embed)

### The attention head

In [62]:
head_size = 7
channels = 3

tril = torch.tril(torch.ones(context_length, context_length))

key = nn.Linear(channels, head_size, bias=False)
query = nn.Linear(channels, head_size, bias=False)
## add in 
value = nn.Linear(channels, head_size, bias=False)

In [61]:
x = torch.randn(batch_size, context_length, channels)

k = key(x)
# k = B, T, head_size  - 4, 8, 7
print(k.shape)
q = query(x)
# k = B, T, head_size
print(q.T.shape)
print(q.T.transpose(-2, -1).shape)

#the dot product tells us how much our query matches our key
kq_match = k @ q.transpose(-2,-1) * head_size**-0.5

torch.Size([4, 8, 7])
torch.Size([7, 8, 4])
torch.Size([7, 4, 8])


In [68]:
masked_affin = kq_match.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(masked_affin, 1)

wei[0]

v = value(x)
out = wei @ v
out[0], out.shape

(tensor([[ 0.0133,  0.0128,  0.0804,  0.0109, -0.0140,  0.0084, -0.0427],
         [-0.0201,  0.0575,  0.1769, -0.0321,  0.0267,  0.0577, -0.1006],
         [ 0.0136,  0.0729,  0.1439, -0.0098,  0.0543, -0.0789,  0.0091],
         [ 0.0576, -0.0090,  0.0178,  0.0561, -0.0350, -0.0893,  0.0386],
         [ 0.0105,  0.0810,  0.1299, -0.0166,  0.0727, -0.1026,  0.0359],
         [-0.2225,  0.1369, -0.0745, -0.2581,  0.2942,  0.1025,  0.0596],
         [-0.2247,  0.1050,  0.0559, -0.2405,  0.1968,  0.2901, -0.1451],
         [-0.1254,  0.0277,  0.1586, -0.1153,  0.0143,  0.3401, -0.2778]],
        grad_fn=<SelectBackward0>),
 torch.Size([4, 8, 7]))